# Polars 入门

![Polars](https://raw.githubusercontent.com/pola-rs/polars-static/master/banner/polars_github_banner.svg)

`Polars` 是一个用于数据操作的 Rust 库，它提供了类似于 `Pandas` 的 API，但是比 `Pandas` `更快。Polars` 也提供了其他运行时的包，可以在 Python/NodeJS 中使用 `Polars`。

在这个笔记本中，我们将主要学习如何在 Python 中使用 `Polars`。NodeJS 接口基本一致，可以

## 安装

### Python (via PyPI)

```bash
pip install polars
```

### NodeJS (via npm)

```bash
npm install nodejs-polars
```


In [1]:
import polars as pl
import pandas as pd
import dataclasses

## 1. 初始化构造

`Polars` 构造函数的入参中我们需要关注的一般只有两个，`data` 和 `schema`，可以理解为 `Pandas` 构造函数中的 `data` 和 `dtypes`。

其中 `data` 支持 `dict`, `Sequence`, `ndarray`, `Series`, or `pandas.DataFrame`。与 `Pandas` 不同的是不支持从 `records`/`tuple` 构建的方式。


In [2]:
@dataclasses.dataclass
class Record:
    x: float 
    y: int 
    z: str 

pd.DataFrame([Record(x=0.1, y=1, z="hello"), Record(x=0.2, y=2, z="pandas")])

,x,y,z
0,0.1,1,hello
1,0.2,2,pandas


详细的构造函数签名可以参考 `DataFrame.__init__`


In [3]:
print(pl.DataFrame.__doc__)


    Two-dimensional data structure representing data as a table with rows and columns.

    Parameters
    ----------
    data : dict, Sequence, ndarray, Series, or pandas.DataFrame
        Two-dimensional data in various forms; dict input must contain Sequences,
        Generators, or a `range`. Sequence may contain Series or other Sequences.
    schema : Sequence of str, (str,DataType) pairs, or a {str:DataType,} dict
        The schema of the resulting DataFrame. The schema may be declared in several
        ways:

        * As a dict of {name:type} pairs; if type is None, it will be auto-inferred.
        * As a list of column names; in this case types are automatically inferred.
        * As a list of (name,type) pairs; this is equivalent to the dictionary form.

        If you supply a list of column names that does not match the names in the
        underlying data, the names given here will overwrite them. The number
        of names given in the schema should match the underl

In [4]:
# Simple
pl.DataFrame({
    "x": [1 , 2, 3],
    "y": None,
    "z": ["female", "male", "female"],
})

x,y,z
i64,null,str
1,null,"""female"""
2,null,"""male"""
3,null,"""female"""


`schema` 支持 `list[str]` / `list[tuple[str, pl.DataTypeClass]]` / `dict[str, pl.DataTypeClass]`，不支持从 `str` 构建类型

> 某些情况下我们必须要从 `str` 构建类型的话可以采用 `polars` 提供的工具函数 `pl.datatypes.convert.dtype_to_ffiname`，具体见 [工具函数](#PolarsUtils)

需要注意的是 `pl.Categorical` 和 `pl.Enum` 这两个类型，类似于 `pd.Categorical`，这两个类型都是代表分类变量的，但是区别在于

1. `polars` 强制要求 `Categorical`/`Enum` 类型先是字符串类型，否则 `raise`
2. Categorical 和 Enum 存在区别，`Categorical` 在构建时可以不指定分类级别，然而 Enum 必须要指定
3. Categorical 支持通过字符序/出现先后顺序指定大小，Enum 通过指定的级别的索引指定大小


In [5]:
print("pl.Categorical")
print("================")
print(pl.Categorical.__doc__)

print("pl.Enum")
print("================")
print(pl.Enum.__doc__)

pl.Categorical

    A categorical encoding of a set of strings.

    Parameters
    ----------
    ordering : {'lexical', 'physical'}
        Ordering by order of appearance (`'physical'`, default)
        or string value (`'lexical'`).
    
pl.Enum

    A fixed set categorical encoding of a set of strings.

    .. warning::
        This functionality is considered **unstable**.
        It is a work-in-progress feature and may not always work as expected.
        It may be changed at any point without it being considered a breaking change.

    Parameters
    ----------
    categories
        The categories in the dataset. Categories must be strings.
    


In [6]:
pl.DataFrame(
  {
    "SEX": ["F", "M", "M", "F"],
    "ID": [1, 2, 2, 3],
    "WEIGHT": [70.2, 88.8, 49.3, 55],
    "GROUP": ["B", "A", "A", "B"],
  },
  {
    "SEX": pl.Categorical,
    "ID": pl.Int8,
    "WEIGHT": float,
    "GROUP": pl.Enum(["A", "B"]),
  }
)


SEX,ID,WEIGHT,GROUP
cat,i8,f64,enum
"""F""",1,70.2,"""B"""
"""M""",2,88.8,"""A"""
"""M""",2,49.3,"""A"""
"""F""",3,55.0,"""B"""


## 数据处理

`polars` 和 `pandas` 最大的区别（或者说我最关心的差异）是 `polars` 的所有数据处理都会返回一份拷贝，而 `pandas` 会返回视图。这样的设计使得 `polars` 更加安全，同时由于 Rust 原生的 Ownership 机制，使得 `polars` 的性能并不会收到太大影响。同时这样的特性可以有效的帮助开发者避免一些潜在的错误，特别是在一些存在数据处理，但是并不想影响原数据的情况下，如多线程环境，函数式编程，底层算法等。


In [7]:
example = {
    "a": [1, 2, 3, 4, 5],
    "b": [5.5, 4.4, 3.3, 2.2, 1.1],
    "c": ["a", "b", "c", "d", "e"]
}
example_pl = pl.DataFrame(example)
example_pd = pd.DataFrame(example)


In [8]:
def some_func(df: pl.DataFrame | pd.DataFrame) -> None:
    df["a"] = 123

In [9]:
some_func(example_pd)
example_pd # Changed！很危险

,a,b,c
0,123,5.5,a
1,123,4.4,b
2,123,3.3,c
3,123,2.2,d
4,123,1.1,e


In [10]:
%%capture
# 会报错，赞 👍
some_func(example_pl)

TypeError: DataFrame object does not support `Series` assignment by index

Use `DataFrame.with_columns`.

使用 `polars` 数据处理和 R 的 `dplyr` 风格很相似，可以通过 `DataFrame` 的方法链式调用，或者通过 `pl` 模块提供的函数式编程接口。大部分的方法和 SQL 中的操作是一一对应的，如 `select`, `filter`, `groupby`, `join`, `sort`, `agg`, `pivot`, `explode` 等等。具体的方法可以参考 [Python API 文档](https://docs.pola.rs/api/python/stable/reference/dataframe/modify_select.html) 或者 [NodeJS API 文档](https://pola-rs.github.io/nodejs-polars/interfaces/pl.DataFrame-1.html)。本文档会主要介绍一些常用的方法。


## <a id="PolarsUtils">工具函数</a>


1. `pl.DataType` 与 `str` 的互转

`pl.DataType` 由于是 Rust 内的数据结构，如果我们需要一个 Python 对应的序列化/反序列化方式，需要将其转成字符串处理


In [11]:
from polars.datatypes import convert

int64_str = convert.dtype_to_ffiname(pl.Int64)
int64_str

'i64'

In [12]:
int64_dtype = convert.dtype_short_repr_to_dtype(int64_str)
int64_dtype == pl.Int64

True

2. 使用 `Arrow` 的 C 指针

`Polars` 的底层（向量）是通过 Apache Arrow 进行存储的，如果需要访问指针级别的数据可以通过内部接口 `_export_arrow_to_c` / `_import_arrow_from_c` 实现


In [13]:
from pyext import simple
allocator = simple.SimpleAllocator([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
address = allocator.address_of()
simple.test(*address)

Export ok? 1
1
2
3
4
5
6
7
8
9
10


In [14]:
pl.Series("x", [11, 22])._export_arrow_to_c(*address)
simple.test(*address)

11
22
